In [2]:
# Step 1: Fetch data from YouBike API
import requests
youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
res= requests.get(youbike_url)
if res.status_code == 200:
    print('下載成功')
else:
    print('下載失敗')

下載成功


In [16]:
from pydantic import BaseModel, Field, field_validator,RootModel

# Step 2: Define Pydantic models
class Bike(BaseModel):

    station: str = Field(alias='sna')
    station_area:str=Field(alias='sarea')
    time:str=Field(alias='mday')
    address:str=Field(alias='ar')
    status:bool=Field(alias='act')
    total:int
    borrow:int=Field(alias='available_rent_bikes')
    latitude:float
    longitude:float
    return_bikes:int=Field(alias='available_return_bikes')

    @field_validator('station')
    @classmethod
    def get_name(cls, value: str) -> str:
        return value.split('_')[-1]

class YouBike(RootModel):
    root: list[Bike]
    def __iter__(self):
        return iter(self.root)
    
    def __getitem__(self, item):
        return self.root[item]

# Step 3: Validate and parse JSON data
data: YouBike = YouBike.model_validate_json(res.text)
all_sites: list[dict] = data.model_dump()


# Step 4: Create a DataFrame
import pandas as pd

df1 = pd.DataFrame(all_sites)
df1

,station,station_area,time,address,status,total,borrow,latitude,longitude,return_bikes
0,捷運科技大樓站,大安區,2024-06-16 19:46:19,復興南路二段235號前,True,28,8,25.02605,121.54360,20
1,復興南路二段273號前,大安區,2024-06-16 19:46:19,復興南路二段273號西側,True,21,4,25.02565,121.54357,17
2,國北教大實小東側門,大安區,2024-06-16 19:13:20,和平東路二段96巷7號,True,16,5,25.02429,121.54124,11
3,和平公園東側,大安區,2024-06-16 19:21:21,和平東路二段118巷33號,True,11,8,25.02351,121.54282,3
4,辛亥復興路口西北側,大安區,2024-06-16 19:44:20,復興南路二段368號,True,16,9,25.02153,121.54299,7
...,...,...,...,...,...,...,...,...,...,...
1418,臺大總圖書館西南側,臺大公館校區,2024-06-16 19:16:19,臺大圖書館西南側,True,30,21,25.01690,121.54031,9
1419,臺大黑森林西側,臺大公館校區,2024-06-16 19:41:20,臺大霖澤館南側,True,20,15,25.01995,121.54347,5
1420,臺大獸醫館南側,臺大公館校區,2024-06-16 19:40:29,臺大獸醫系館南側,True,24,12,25.01791,121.54242,12
1421,臺大新體育館東南側,臺大公館校區,2024-06-16 19:38:20,臺大體育館東側,True,40,1,25.02112,121.53591,39


In [10]:

from pydantic import BaseModel, Field, RootModel, field_validator

class Site(BaseModel):
    站名: str = Field(alias='sna')
    區域: str = Field(alias='sarea')
    時間: str = Field(alias='mday')
    地點: str = Field(alias='ar')
    總車輛數: int = Field(alias='total')
    可借數量: int = Field(alias='available_rent_bikes')
    可還數量: int = Field(alias='available_return_bikes')
    經度: float = Field(alias='longitude')
    緯度: float = Field(alias='latitude')
    狀態: bool = Field(alias='act')

    @field_validator('站名')
    @classmethod
    def get_name(cls, value: str) -> str:
        return value.split('_')[-1]

class YouBike(RootModel):
    root: list[Site]
    def __iter__(self):
        return iter(self.root)
    
    def __getitem__(self, item):
        return self.root[item]
    
data: YouBike = YouBike.model_validate_json(res.text)
all_sites: list[dict] = data.model_dump()

import pandas as pd

df1 = pd.DataFrame(all_sites)
df1

,站名,區域,時間,地點,總車輛數,可借數量,可還數量,經度,緯度,狀態
0,捷運科技大樓站,大安區,2024-06-16 19:46:19,復興南路二段235號前,28,8,20,121.54360,25.02605,True
1,復興南路二段273號前,大安區,2024-06-16 19:46:19,復興南路二段273號西側,21,4,17,121.54357,25.02565,True
2,國北教大實小東側門,大安區,2024-06-16 19:13:20,和平東路二段96巷7號,16,5,11,121.54124,25.02429,True
3,和平公園東側,大安區,2024-06-16 19:21:21,和平東路二段118巷33號,11,8,3,121.54282,25.02351,True
4,辛亥復興路口西北側,大安區,2024-06-16 19:44:20,復興南路二段368號,16,9,7,121.54299,25.02153,True
...,...,...,...,...,...,...,...,...,...,...
1418,臺大總圖書館西南側,臺大公館校區,2024-06-16 19:16:19,臺大圖書館西南側,30,21,9,121.54031,25.01690,True
1419,臺大黑森林西側,臺大公館校區,2024-06-16 19:41:20,臺大霖澤館南側,20,15,5,121.54347,25.01995,True
1420,臺大獸醫館南側,臺大公館校區,2024-06-16 19:40:29,臺大獸醫系館南側,24,12,12,121.54242,25.01791,True
1421,臺大新體育館東南側,臺大公館校區,2024-06-16 19:38:20,臺大體育館東側,40,1,39,121.53591,25.02112,True
